## Extracción del texto

In [1]:
import fitz 

In [2]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_path = "./fiodor.pdf"
raw_text = extract_text_from_pdf(pdf_path)


In [3]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\n+', '\n', text)  # Unifica saltos de línea
    text = re.sub(r'\s+', ' ', text)  # Elimina espacios extras
    text = re.sub(r'\d+\s*', '', text)  # Elimina números de página u otros
    text = text.strip()
    return text

cleaned_text = clean_text(raw_text)

In [15]:
cleaned_text[:1000]

'crimen y castigo por fedor mikhaïlovitch dostoïevski parte capítulo una tarde extremadamente calurosa de principios de julio, un joven salió de la reducida habitación que tenía alquilada en la callejuela de s y, con paso lento e indeciso, se dirigió al puente k. había tenido la suerte de no encontrarse con su patrona en la escalera. su cuartucho se hallaba bajo el tejado de un gran edificio de cinco pisos y, más que una habitación, parecía una alacena. en cuanto a la patrona, que le había alquilado el cuarto con servicio y pensión, ocupaba un departamento del piso de abajo; de modo que nuestro joven, cada vez que salía, se veía obligado a pasar por delante de la puerta de la cocina, que daba a la escalera y estaba casi siempre abierta de par en par. en esos momentos experimentaba invariablemente una sensación ingrata de vago temor, que le humillaba y daba a su semblante una expresión sombría. debía una cantidad considerable a la patrona y por eso temía encontrarse con ella. no es que 

## Crear secuencias de texto para entrenamiento
Necesitamos convertir el texto en secuencias que la red LSTM pueda usar. Por ejemplo, para cada 40 caracteres, predecir el siguiente.

In [5]:
seq_length = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(cleaned_text) - seq_length, step):
    sentences.append(cleaned_text[i: i + seq_length])
    next_chars.append(cleaned_text[i + seq_length])


In [17]:
sentences[:10]

['crimen y castigo por fedor mikhaïlovitch',
 'men y castigo por fedor mikhaïlovitch do',
 ' y castigo por fedor mikhaïlovitch dosto',
 'castigo por fedor mikhaïlovitch dostoïev',
 'tigo por fedor mikhaïlovitch dostoïevski',
 'o por fedor mikhaïlovitch dostoïevski pa',
 'or fedor mikhaïlovitch dostoïevski parte',
 'fedor mikhaïlovitch dostoïevski parte ca',
 'or mikhaïlovitch dostoïevski parte capít',
 'mikhaïlovitch dostoïevski parte capítulo']

In [20]:
 next_chars[:10]

[' ', 's', 'ï', 's', ' ', 'r', ' ', 'p', 'u', ' ']

## Vectorizar el texto
Convertimos los caracteres a vectores one-hot.

In [6]:
import numpy as np
chars = sorted(list(set(cleaned_text)))
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

X = np.zeros((len(sentences), seq_length, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


In [22]:
X.shape

(403279, 40, 56)

In [32]:
X[0][0:3][:]

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, Fa

## Crear el modelo LSTM en Keras

In [7]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')


2025-04-07 14:55:54.308131: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
//anaconda3/envs/tareaDL/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        94,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 56)             │         7,224 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,944 (398.22 KB)

 Trainable params: 101,944 (398.22 KB)

 Non-trainable params: 0 (0.00 B)

## Entrenar

In [9]:
model.fit(X, y, batch_size=128, epochs=20)


Epoch 1/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 144s 45ms/step - loss: 2.5489
Epoch 2/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 126s 40ms/step - loss: 1.9590
Epoch 3/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 126s 40ms/step - loss: 1.8139
Epoch 4/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 126s 40ms/step - loss: 1.7083
Epoch 5/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 128s 41ms/step - loss: 1.6320
Epoch 6/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 129s 41ms/step - loss: 1.5761
Epoch 7/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 129s 41ms/step - loss: 1.5241
Epoch 8/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 130s 41ms/step - loss: 1.4872
Epoch 9/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 131s 42ms/step - loss: 1.4590
Epoch 10/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 133s 42ms/step - loss: 1.4324
Epoch 11/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 154s 49ms/step - loss: 1.4050
Epoch 12/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 161s 51ms/step - loss: 1.3910
Epoch 13/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 167s 53ms/step - loss: 1.3695
Epoch 14/20
3151/3151 ━━━━━━━━━━━━━━━━━━━━ 155s 49ms/step - 

In [10]:
model.save("./lstm1.keras")

In [33]:
len(chars)

56

In [40]:
frase = "quiero encontrar algo que valga la pena "
print(len(frase))

40


In [73]:
def autoregresion(frase_inicial):
    frase_oh = np.zeros((seq_length, len(chars)), dtype=np.bool_)
    for t, char in enumerate(frase_inicial):
        frase_oh[t, char_indices[char]] = 1
    frase_oh = np.expand_dims(frase_oh,axis = 0)
    result = model.predict(frase_oh)
    return indices_char[np.argmax(result[0])]

In [80]:
pred_word = ""
for i in range(5):
    nueva_letra = autoregresion(frase)
    print(nueva_letra)
    frase = frase[1:]
    frase+= nueva_letra
    pred_word+= nueva_letra
print(pred_word)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
o
aia o


# Usando tokenizador

In [4]:
lineas = cleaned_text.split('\n')

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2025-04-07 19:29:18.589335: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Crear y entrenar el tokenizador
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lineas)

#Ver el vocabulario
print(tokenizer.word_index)

{'de': 1, 'que': 2, 'la': 3, 'a': 4, 'y': 5, 'en': 6, 'el': 7, 'se': 8, 'no': 9, 'un': 10, 'su': 11, 'una': 12, 'lo': 13, 'con': 14, 'le': 15, 'por': 16, 'los': 17, 'me': 18, 'es': 19, 'usted': 20, 'había': 21, 'al': 22, 'las': 23, 'pero': 24, 'para': 25, 'raskolnikof': 26, 'ha': 27, 'del': 28, 'como': 29, 'más': 30, 'sus': 31, 'él': 32, 'todo': 33, 'yo': 34, 'sin': 35, 'he': 36, 'si': 37, 'era': 38, 'mi': 39, 'ya': 40, 'qué': 41, 'esto': 42, 'estaba': 43, 'cuando': 44, 'te': 45, 'sonia': 46, 'ella': 47, 'este': 48, 'casa': 49, 'esta': 50, 'vez': 51, 'hombre': 52, 'tan': 53, 'dos': 54, 'momento': 55, '—dijo': 56, 'nada': 57, 'pues': 58, 'sólo': 59, 'sobre': 60, 'después': 61, 'rasumikhine': 62, 'ni': 63, 'está': 64, '¿qué': 65, 'así': 66, '»': 67, 'muy': 68, 'aquí': 69, 'tenía': 70, 'mismo': 71, 'o': 72, 'ivanovna': 73, 'ahora': 74, 'habría': 75, 'petrovitch': 76, 'puerta': 77, 'fin': 78, 'joven': 79, 'todos': 80, 'eso': 81, 'también': 82, 'pronto': 83, 'bien': 84, 'tiene': 85, 'modo':

In [7]:
len(tokenizer.word_index)

19367

In [18]:
new_texto = lineas[0].split(" ")

In [19]:
new_texto

['crimen',
 'y',
 'castigo',
 'por',
 'fedor',
 'mikhaïlovitch',
 'dostoïevski',
 'parte',
 'capítulo',
 'una',
 'tarde',
 'extremadamente',
 'calurosa',
 'de',
 'principios',
 'de',
 'julio,',
 'un',
 'joven',
 'salió',
 'de',
 'la',
 'reducida',
 'habitación',
 'que',
 'tenía',
 'alquilada',
 'en',
 'la',
 'callejuela',
 'de',
 's',
 'y,',
 'con',
 'paso',
 'lento',
 'e',
 'indeciso,',
 'se',
 'dirigió',
 'al',
 'puente',
 'k.',
 'había',
 'tenido',
 'la',
 'suerte',
 'de',
 'no',
 'encontrarse',
 'con',
 'su',
 'patrona',
 'en',
 'la',
 'escalera.',
 'su',
 'cuartucho',
 'se',
 'hallaba',
 'bajo',
 'el',
 'tejado',
 'de',
 'un',
 'gran',
 'edificio',
 'de',
 'cinco',
 'pisos',
 'y,',
 'más',
 'que',
 'una',
 'habitación,',
 'parecía',
 'una',
 'alacena.',
 'en',
 'cuanto',
 'a',
 'la',
 'patrona,',
 'que',
 'le',
 'había',
 'alquilado',
 'el',
 'cuarto',
 'con',
 'servicio',
 'y',
 'pensión,',
 'ocupaba',
 'un',
 'departamento',
 'del',
 'piso',
 'de',
 'abajo;',
 'de',
 'modo',
 'q

In [30]:
tokens = tokenizer.texts_to_sequences(new_texto[0:10])

In [31]:
new_texto[0:10]

['crimen',
 'y',
 'castigo',
 'por',
 'fedor',
 'mikhaïlovitch',
 'dostoïevski',
 'parte',
 'capítulo',
 'una']

In [88]:
tokens

[[4281], [1204], [16], [200], [2], [8], [10750], [6], [213], []]

In [33]:
len(new_texto)

206051

In [86]:
# hacer secuancias de 10 palabras
input_sequences = np.zeros((len(new_texto)-11,10))

for i in range(len(new_texto)-11):
    tokens = tokenizer.texts_to_sequences(new_texto[i:i+10])
    for j in range(10):
        input_sequences[i][j] = tokens[j][0]

IndexError: list index out of range

In [79]:
flat = [[elem[0] for elem in secuencia] for secuencia in input_sequences]

IndexError: list index out of range

In [70]:
input_sequences = np.array(input_sequences)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (206040, 10) + inhomogeneous part.

In [67]:
input_sequences.reshape(len(input_sequences)/10,10)

AttributeError: 'list' object has no attribute 'reshape'

In [43]:
import numpy as np

In [62]:
input_sequences = np.array(input_sequences)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (206040, 10) + inhomogeneous part.

In [ ]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# Convertir etiquetas a one-hot
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=total_palabras)

In [93]:
total_palabras = len(tokenizer.word_index) + 1

In [94]:
total_palabras

45

In [99]:
max_len = max(len(x) for x in secuencias_padded)

In [96]:
X = secuencias_padded[:,:-1]
y = secuencias_padded[:,-1]

In [97]:
# Convertir etiquetas a one-hot
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=total_palabras)

In [98]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

modelo = Sequential()
modelo.add(Embedding(input_dim=total_palabras, output_dim=100, input_length=max_len - 1))
modelo.add(LSTM(150, return_sequences=True))
modelo.add(LSTM(100))
modelo.add(Dense(total_palabras, activation='softmax'))

modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
modelo.summary()

modelo.fit(X, y, epochs=50, verbose=1)


NameError: name 'max_len' is not defined